In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import mudata as md
import seaborn as sns
import matplotlib.pyplot as plt

import gc
import sys
sys.path.append('../')

from tqdm import tqdm
from latentcor import latentcor
from scipy.sparse import csr_matrix
from utils.gglasso_pipeline import gg_lasso_network_analysis
from utils.utils import calc_sparsity, fetch_protein_names

import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

In [2]:
path_raw_data = "../data/raw/"
experiments = ["MGH66_Abx3_P1_align2._CDS.tsv",
               "MGH66_Abx3_P2_align2._CDS.tsv",
               "MGH66_Abx3_P3_align2._CDS.tsv",
               "MGH66_Abx3_P4_align2._CDS.tsv"]

# P1

In [3]:
adata = sc.read_csv(filename=path_raw_data + experiments[0], delimiter='\t')

In [10]:
obs_tmp = adata.var.copy()
var_tmp = adata.obs.copy()
X_tmp = csr_matrix(adata.X.T)

adata = sc.AnnData(X=X_tmp, obs=obs_tmp, var=var_tmp)
adata

In [19]:
print("Sparity:\t\t", calc_sparsity(adata.X.A))
print("Mean counts per gene:\t", np.mean(np.sum(adata.X.A, axis=1)))
print("Median counts per gene:\t", np.median(np.sum(adata.X.A, axis=1)))
print("Mean genes per cell: \t", np.mean(np.sum(adata.X.A, axis=0)))
print("Median genes per cell:\t", np.median(np.sum(adata.X.A, axis=0)))

Sparity:		 0.9980492107440664
Mean counts per gene:	 8.321548
Median counts per gene:	 6.0
Mean genes per cell: 	 839.40607
Median genes per cell:	 97.5


In [ ]:
adata.var["gene"] = [gene[4:] for gene in adata.var.index]
adata.var

In [158]:
#accession_numbers = [gene[4:] for gene in adata.var.index[-10:]]
#accession_numbers = [gene[4:] for gene in adata.var[~adata.var.index.str.startswith("cds-WP")].index.values]
accession_numbers = [gene[4:] for gene in adata.var[adata.var.index.str.startswith("cds-WP")].index.values]
#accession_numbers = adata.var.index.values

In [159]:
len(accession_numbers)

3881

In [ ]:
batch_size = 1
gene_to_protein = dict()

for i in tqdm(range(0, len(accession_numbers), batch_size)):
    try:
        batch = accession_numbers[i:i + batch_size]
        protein_names = fetch_protein_names(batch)
        gene_to_protein.update(dict(zip(batch, protein_names)))
        print(len(gene_to_protein))
    except Exception as e:
        print(f"Error fetching batch {batch}: {e}")
        continue

In [129]:
adata.var['long_protein_name'] = adata.var['gene'].map(gene_to_protein)
adata.var['long_protein_name'].fillna("None", inplace=True)
adata.var

,gene,long_protein_name
cds-AF52_RS00180,AF52_RS00180,None
cds-AF52_RS00275,AF52_RS00275,None
cds-AF52_RS00620,AF52_RS00620,None
cds-AF52_RS01000,AF52_RS01000,None
cds-AF52_RS01015,AF52_RS01015,None
...,...,...
cds-WP_187079192.1,WP_187079192.1,WP_187079192.1 IS5-like element IS903B family ...
cds-WP_187079193.1,WP_187079193.1,WP_187079193.1 aspartate aminotransferase fami...
cds-WP_187079194.1,WP_187079194.1,WP_187079194.1 ribokinase [Klebsiella pneumoniae]
cds-WP_187079196.1,WP_187079196.1,WP_187079196.1 6-carboxytetrahydropterin synth...


In [149]:
adata.var['species'] = adata.var['long_protein_name'].str.extract(r'\[([^]]+)\]$')
adata.var.species.value_counts()

Klebsiella                     2399
Klebsiella pneumoniae           769
Enterobacteriaceae              324
Gammaproteobacteria             126
Enterobacterales                 70
Bacteria                         54
Klebsiella/Raoultella group      22
Pseudomonadota                    7
Klebsiella quasipneumoniae        2
Klebsiella grimontii              1
Name: species, dtype: int64

In [150]:
adata.var['protein_name'] = adata.var['long_protein_name'].str.split(' ', 1).str[1].str.rsplit('[', 1).str[0].str.strip()
adata.var

,gene,long_protein_name,species,protein_name
cds-AF52_RS00180,AF52_RS00180,None,NaN,NaN
cds-AF52_RS00275,AF52_RS00275,None,NaN,NaN
cds-AF52_RS00620,AF52_RS00620,None,NaN,NaN
cds-AF52_RS01000,AF52_RS01000,None,NaN,NaN
cds-AF52_RS01015,AF52_RS01015,None,NaN,NaN
...,...,...,...,...
cds-WP_187079192.1,WP_187079192.1,WP_187079192.1 IS5-like element IS903B family ...,Klebsiella pneumoniae,IS5-like element IS903B family transposase
cds-WP_187079193.1,WP_187079193.1,WP_187079193.1 aspartate aminotransferase fami...,Klebsiella pneumoniae,aspartate aminotransferase family protein
cds-WP_187079194.1,WP_187079194.1,WP_187079194.1 ribokinase [Klebsiella pneumoniae],Klebsiella pneumoniae,ribokinase
cds-WP_187079196.1,WP_187079196.1,WP_187079196.1 6-carboxytetrahydropterin synth...,Klebsiella pneumoniae,6-carboxytetrahydropterin synthase QueD


In [152]:
adata.var['rRNA'] = adata.var['long_protein_name'].str.contains(r'30S|50S')
adata.var["rRNA"].value_counts()

False    4005
True       61
Name: rRNA, dtype: int64

In [168]:
adata

AnnData object with n_obs × n_vars = 410143 × 4066
    var: 'gene', 'long_protein_name', 'species', 'protein_name', 'rRNA'

In [167]:
sc.write(adata=adata, filename="../data/preprocessed/MGH66_P1_untreated.h5ad")

# P2

In [3]:
adata = sc.read_csv(filename=path_raw_data + experiments[1], delimiter='\t')

In [4]:
adata

AnnData object with n_obs × n_vars = 3987 × 392941

In [5]:
obs_tmp = adata.var.copy()
var_tmp = adata.obs.copy()
X_tmp = csr_matrix(adata.X.T)

adata = sc.AnnData(X=X_tmp, obs=obs_tmp, var=var_tmp)
adata

AnnData object with n_obs × n_vars = 392941 × 3987

In [6]:
adata.var

""
cds-AF52_RS00165
cds-AF52_RS00180
cds-AF52_RS00620
cds-AF52_RS01000
cds-AF52_RS01010
...
cds-WP_187079193.1
cds-WP_187079194.1
cds-WP_187079195.1
cds-WP_187079196.1


In [7]:
adata.var["gene"] = [gene[4:] for gene in adata.var.index]
adata.var

,gene
cds-AF52_RS00165,AF52_RS00165
cds-AF52_RS00180,AF52_RS00180
cds-AF52_RS00620,AF52_RS00620
cds-AF52_RS01000,AF52_RS01000
cds-AF52_RS01010,AF52_RS01010
...,...
cds-WP_187079193.1,WP_187079193.1
cds-WP_187079194.1,WP_187079194.1
cds-WP_187079195.1,WP_187079195.1
cds-WP_187079196.1,WP_187079196.1


In [8]:
accession_numbers = [gene[4:] for gene in adata.var[adata.var.index.str.startswith("cds-WP")].index.values]
len(accession_numbers)

3800

In [9]:
batch_size = 200
gene_to_protein = dict()

for i in tqdm(range(0, len(accession_numbers), batch_size)):
    try:
        batch = accession_numbers[i:i + batch_size]
        protein_names = fetch_protein_names(batch)
        gene_to_protein.update(dict(zip(batch, protein_names)))
        print(len(gene_to_protein))
    except Exception as e:
        print(f"Error fetching batch {batch}: {e}")
        continue

  5%|▌         | 1/19 [00:03<01:05,  3.64s/it]

200


 11%|█         | 2/19 [00:08<01:15,  4.43s/it]

400


 16%|█▌        | 3/19 [00:14<01:17,  4.86s/it]

...

3233


 95%|█████████▍| 18/19 [00:57<00:02,  2.93s/it]

3433


100%|██████████| 19/19 [01:05<00:00,  3.44s/it]

3633


In [11]:
adata.var['long_protein_name'] = adata.var['gene'].map(gene_to_protein)
adata.var['long_protein_name'].fillna("None", inplace=True)
adata.var

,gene,long_protein_name
cds-AF52_RS00165,AF52_RS00165,None
cds-AF52_RS00180,AF52_RS00180,None
cds-AF52_RS00620,AF52_RS00620,None
cds-AF52_RS01000,AF52_RS01000,None
cds-AF52_RS01010,AF52_RS01010,None
...,...,...
cds-WP_187079193.1,WP_187079193.1,WP_187079193.1 aspartate aminotransferase fami...
cds-WP_187079194.1,WP_187079194.1,WP_187079194.1 ribokinase [Klebsiella pneumoniae]
cds-WP_187079195.1,WP_187079195.1,WP_187079195.1 HlyD family efflux transporter ...
cds-WP_187079196.1,WP_187079196.1,WP_187079196.1 6-carboxytetrahydropterin synth...


In [12]:
adata.var['species'] = adata.var['long_protein_name'].str.extract(r'\[([^]]+)\]$')
adata.var.species.value_counts()

Klebsiella                     2291
Klebsiella pneumoniae           741
Enterobacteriaceae              332
Gammaproteobacteria             121
Enterobacterales                 67
Bacteria                         54
Klebsiella/Raoultella group      15
Pseudomonadota                    9
Klebsiella quasipneumoniae        2
Klebsiella grimontii              1
Name: species, dtype: int64

In [13]:
adata.var['protein_name'] = adata.var['long_protein_name'].str.split(' ', 1).str[1].str.rsplit('[', 1).str[0].str.strip()
adata.var

,gene,long_protein_name,species,protein_name
cds-AF52_RS00165,AF52_RS00165,None,NaN,NaN
cds-AF52_RS00180,AF52_RS00180,None,NaN,NaN
cds-AF52_RS00620,AF52_RS00620,None,NaN,NaN
cds-AF52_RS01000,AF52_RS01000,None,NaN,NaN
cds-AF52_RS01010,AF52_RS01010,None,NaN,NaN
...,...,...,...,...
cds-WP_187079193.1,WP_187079193.1,WP_187079193.1 aspartate aminotransferase fami...,Klebsiella pneumoniae,aspartate aminotransferase family protein
cds-WP_187079194.1,WP_187079194.1,WP_187079194.1 ribokinase [Klebsiella pneumoniae],Klebsiella pneumoniae,ribokinase
cds-WP_187079195.1,WP_187079195.1,WP_187079195.1 HlyD family efflux transporter ...,Klebsiella pneumoniae,HlyD family efflux transporter periplasmic ada...
cds-WP_187079196.1,WP_187079196.1,WP_187079196.1 6-carboxytetrahydropterin synth...,Klebsiella pneumoniae,6-carboxytetrahydropterin synthase QueD


In [17]:
adata.var['rRNA'] = adata.var['long_protein_name'].str.contains(r'30S|50S')
adata.var["rRNA"].value_counts()

False    3926
True       61
Name: rRNA, dtype: int64

In [18]:
sc.write(adata=adata, filename="../data/preprocessed/MGH66_P2_meropenem.h5ad")

# P3

In [3]:
adata = sc.read_csv(filename=path_raw_data + experiments[2], delimiter='\t')

In [4]:
adata

AnnData object with n_obs × n_vars = 3391 × 390322

In [5]:
obs_tmp = adata.var.copy()
var_tmp = adata.obs.copy()
X_tmp = csr_matrix(adata.X.T)

adata = sc.AnnData(X=X_tmp, obs=obs_tmp, var=var_tmp)
adata

AnnData object with n_obs × n_vars = 390322 × 3391

In [6]:
adata.var["gene"] = [gene[4:] for gene in adata.var.index]
adata.var

,gene
cds-AF52_RS00165,AF52_RS00165
cds-AF52_RS00275,AF52_RS00275
cds-AF52_RS00620,AF52_RS00620
cds-AF52_RS01000,AF52_RS01000
cds-AF52_RS01010,AF52_RS01010
...,...
cds-WP_187079193.1,WP_187079193.1
cds-WP_187079194.1,WP_187079194.1
cds-WP_187079195.1,WP_187079195.1
cds-WP_187079196.1,WP_187079196.1


In [7]:
accession_numbers = [gene[4:] for gene in adata.var[adata.var.index.str.startswith("cds-WP")].index.values]
len(accession_numbers)

3246

In [8]:
batch_size = 200
gene_to_protein = dict()

for i in tqdm(range(0, len(accession_numbers), batch_size)):
    try:
        batch = accession_numbers[i:i + batch_size]
        protein_names = fetch_protein_names(batch)
        gene_to_protein.update(dict(zip(batch, protein_names)))
        print(len(gene_to_protein))
    except Exception as e:
        print(f"Error fetching batch {batch}: {e}")
        continue

  6%|▌         | 1/17 [00:03<00:57,  3.56s/it]

200


 12%|█▏        | 2/17 [00:07<01:00,  4.05s/it]

400


 18%|█▊        | 3/17 [00:11<00:52,  3.75s/it]

...

2909


 94%|█████████▍| 16/17 [01:03<00:04,  4.50s/it]

3109


100%|██████████| 17/17 [01:07<00:00,  3.99s/it]

3155


In [9]:
adata.var['long_protein_name'] = adata.var['gene'].map(gene_to_protein)
adata.var['long_protein_name'].fillna("None", inplace=True)

adata.var['species'] = adata.var['long_protein_name'].str.extract(r'\[([^]]+)\]$')

adata.var['protein_name'] = adata.var['long_protein_name'].str.split(' ', 1).str[1].str.rsplit('[', 1).str[0].str.strip()

adata.var['rRNA'] = adata.var['long_protein_name'].str.contains(r'30S|50S')

adata.var

,gene,long_protein_name,species,protein_name,rRNA
cds-AF52_RS00165,AF52_RS00165,None,NaN,NaN,False
cds-AF52_RS00275,AF52_RS00275,None,NaN,NaN,False
cds-AF52_RS00620,AF52_RS00620,None,NaN,NaN,False
cds-AF52_RS01000,AF52_RS01000,None,NaN,NaN,False
cds-AF52_RS01010,AF52_RS01010,None,NaN,NaN,False
...,...,...,...,...,...
cds-WP_187079193.1,WP_187079193.1,WP_187079193.1 aspartate aminotransferase fami...,Klebsiella pneumoniae,aspartate aminotransferase family protein,False
cds-WP_187079194.1,WP_187079194.1,WP_187079194.1 ribokinase [Klebsiella pneumoniae],Klebsiella pneumoniae,ribokinase,False
cds-WP_187079195.1,WP_187079195.1,WP_187079195.1 HlyD family efflux transporter ...,Klebsiella pneumoniae,HlyD family efflux transporter periplasmic ada...,False
cds-WP_187079196.1,WP_187079196.1,WP_187079196.1 6-carboxytetrahydropterin synth...,Klebsiella pneumoniae,6-carboxytetrahydropterin synthase QueD,False


In [10]:
sc.write(adata=adata, filename="../data/preprocessed/MGH66_P3_ciprofloxacin.h5ad")

# P4

In [3]:
adata = sc.read_csv(filename=path_raw_data + experiments[0], delimiter='\t')

In [4]:
obs_tmp = adata.var.copy()
var_tmp = adata.obs.copy()
X_tmp = csr_matrix(adata.X.T)

adata = sc.AnnData(X=X_tmp, obs=obs_tmp, var=var_tmp)
adata

AnnData object with n_obs × n_vars = 410143 × 4066

In [5]:
adata.var["gene"] = [gene[4:] for gene in adata.var.index]
adata.var

,gene
cds-AF52_RS00180,AF52_RS00180
cds-AF52_RS00275,AF52_RS00275
cds-AF52_RS00620,AF52_RS00620
cds-AF52_RS01000,AF52_RS01000
cds-AF52_RS01015,AF52_RS01015
...,...
cds-WP_187079192.1,WP_187079192.1
cds-WP_187079193.1,WP_187079193.1
cds-WP_187079194.1,WP_187079194.1
cds-WP_187079196.1,WP_187079196.1


In [6]:
accession_numbers = [gene[4:] for gene in adata.var[adata.var.index.str.startswith("cds-WP")].index.values]
len(accession_numbers)

3881

In [7]:
batch_size = 200
gene_to_protein = dict()

for i in tqdm(range(0, len(accession_numbers), batch_size)):
    try:
        batch = accession_numbers[i:i + batch_size]
        protein_names = fetch_protein_names(batch)
        gene_to_protein.update(dict(zip(batch, protein_names)))
        print(len(gene_to_protein))
    except Exception as e:
        print(f"Error fetching batch {batch}: {e}")
        continue

  5%|▌         | 1/20 [00:03<01:09,  3.64s/it]

200


 10%|█         | 2/20 [00:08<01:14,  4.15s/it]

400


 15%|█▌        | 3/20 [00:12<01:12,  4.27s/it]

...

3493


 95%|█████████▌| 19/20 [02:07<00:08,  8.65s/it]

3693


100%|██████████| 20/20 [02:12<00:00,  6.62s/it]

3774


In [8]:
adata.var['long_protein_name'] = adata.var['gene'].map(gene_to_protein)
adata.var['long_protein_name'].fillna("None", inplace=True)

adata.var['species'] = adata.var['long_protein_name'].str.extract(r'\[([^]]+)\]$')

adata.var['protein_name'] = adata.var['long_protein_name'].str.split(' ', 1).str[1].str.rsplit('[', 1).str[0].str.strip()

adata.var['rRNA'] = adata.var['long_protein_name'].str.contains(r'30S|50S')

adata.var

,gene,long_protein_name,species,protein_name,rRNA
cds-AF52_RS00180,AF52_RS00180,None,NaN,NaN,False
cds-AF52_RS00275,AF52_RS00275,None,NaN,NaN,False
cds-AF52_RS00620,AF52_RS00620,None,NaN,NaN,False
cds-AF52_RS01000,AF52_RS01000,None,NaN,NaN,False
cds-AF52_RS01015,AF52_RS01015,None,NaN,NaN,False
...,...,...,...,...,...
cds-WP_187079192.1,WP_187079192.1,WP_187079192.1 IS5-like element IS903B family ...,Klebsiella pneumoniae,IS5-like element IS903B family transposase,False
cds-WP_187079193.1,WP_187079193.1,WP_187079193.1 aspartate aminotransferase fami...,Klebsiella pneumoniae,aspartate aminotransferase family protein,False
cds-WP_187079194.1,WP_187079194.1,WP_187079194.1 ribokinase [Klebsiella pneumoniae],Klebsiella pneumoniae,ribokinase,False
cds-WP_187079196.1,WP_187079196.1,WP_187079196.1 6-carboxytetrahydropterin synth...,Klebsiella pneumoniae,6-carboxytetrahydropterin synthase QueD,False


In [9]:
sc.write(adata=adata, filename="../data/preprocessed/MGH66_P4_gentamicin.h5ad")